In [1]:
import random
import os
import torch
import torchaudio
import torch.nn.functional as F
import nbimporter
from torch.utils.data import Dataset, DataLoader
import pickle
import sys
import pandas as pd

import torch.nn as nn
from torch import optim



In [2]:

import matplotlib.pyplot as plt
import pylab


In [3]:
# Para una visualización más interactiva defino esta función que básicamente tiene todo lo que me gusta en cuanto a gráficas.

def set_default(figsize=(10, 10), dpi=100):
    plt.style.use([ 'dark_background','bmh'])
    plt.rc('axes', facecolor='k')
    plt.rc('figure', facecolor='k')
    plt.rc('figure', figsize=figsize, dpi=dpi)
    
    
    

In [4]:
# Definimos el tamaño de las gráficas
set_default(figsize=(16, 8))

In [5]:
#Definiendo parámetros
epocs=400 #epocas a correr

In [7]:
df = pd.read_csv('/media/birds/BirdsDataFrame.csv')
df=df.drop(['Unnamed: 0'], axis=1)
df

especies=df['Species'].unique().tolist()

OSError: [Errno 5] Input/output error: '/media/birds/BirdsDataFrame.csv'

In [ ]:
class Io:
    
        def __init__(self):
            pass
       
        
        def spectrogram(self,audio_path = None):
            waveform, sample_rate = torchaudio.load(audio_path)
           # print("Shape of waveform: {}".format(waveform.size()))
            #print("Sample rate of waveform: {}".format(sample_rate))

            # Plot de waveforms

            # plt.figure()
            # plt.plot(waveform.t().numpy())

            tensor = torchaudio.transforms.Spectrogram()(waveform) #many arguments to modificate the spectrogram 

          #  print("Shape of spectrogram: {}".format(tensor.size()))

            #plt.figure()
            #pylab.matshow(tensor.log2()[0,:,:].numpy(), cmap=pylab.get_cmap('plasma'))

            self.tensor = tensor





In [ ]:
class AvesDataset:
    """Class of dataset"""
    #especies=df['Species'].unique().tolist()

    def __init__(self, nAncla=1,nPos=1, nNeg=1, winSize=50):
        self.nAncla=nAncla
        self.nPos=nPos
        self.nNeg=nNeg
        self.winSize=winSize
        
       
        
   
    def pathRandSpectSam(self, path, winSize):
        
        io = Io()
        io.spectrogram(audio_path=path)
        spectogram = io.tensor
        startIdx = random.randint(0, spectogram.size()[2] - winSize)
        spectogram = spectogram[:, :, startIdx:startIdx+winSize]
        
        return spectogram[0].view(1,-1,winSize)


    def __len__(self, re=epocs):
        self.re=re
        
        return re
    
    def __getitem__(self,iterator):
        #iteration?
        pair=random.sample(especies,2)
        AnclaSpects = []   # Serán del mismo tipo que los positivos
        anchPosSpects = []
        negaSpects=[]
        AnclaPos=df['FileName'][df['Species'] == pair[0]].to_list()
        anchPositive=df['FileName'][df['Species'] == pair[0]].to_list()
        negative=df['FileName'][df['Species'] == pair[1]].to_list()
        AnclaPaths=random.sample(AnclaPos,self.nAncla)
        anchPosPaths=random.sample(anchPositive,self.nPos)
        negaPaths = random.sample(negative, self.nNeg)
        

        for path in AnclaPaths:
            path='/media/birds/data/'+path
            tempSpec = self.pathRandSpectSam(path, self.winSize)
            AnclaSpects.append(tempSpec)
            
        
        for path in anchPosPaths:
            path='/media/birds/data/'+path
            tempSpec = self.pathRandSpectSam(path, self.winSize)
            anchPosSpects.append(tempSpec)

       

        for path in negaPaths:
            path='/media/birds/data/'+path
            tempSpec = self.pathRandSpectSam(path, self.winSize)
            negaSpects.append(tempSpec)

            print(tempSpec.shape)
            
            
        return { 
        'spectograms': {'Ancla': AnclaSpects,
                        'positive': anchPosSpects,
                        'negative': negaSpects
                       },
        'labels': {'Ancla': 1,
                                'positive': 1,
                                   'negative': 0
                  }}
    
# 1 para especies iguales y 0 para diferentes
 

In [ ]:
AvesData = AvesDataset()

In [ ]:
class Net(nn.Module):

    def __init__(self,conv11=1,conv12=3,conv13=5, L1=2115,L12=120,L23=84, L4=10):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv11=conv11
        self.conv12=conv12
        self.conv13=conv13
        self.L1=L1
        self.L12=L12
        self.L23=L23
        self.L4=L4
        
        self.conv1 = nn.Conv2d(conv11, conv12, conv13)
        self.conv2 = nn.Conv2d(conv12, conv13, conv13)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(L1, L12)  # 5*5 from image dimension
        self.fc2 = nn.Linear(L12, L23)
        self.fc3 = nn.Linear(L23, L4)

    def forward(self, x,poo1=2,poo2=2,poo3=2,flat=1):
        self.poo1=poo1
        self.poo2=poo2
        self.poo3=poo3
        self.flat=flat
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (poo1, poo2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), poo3)
        x = torch.flatten(x, flat) # flatten all dimensions except the batch dimension
        print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x




In [ ]:
#Modelo de red neuronal

class SiameseNetwork(nn.Module):
    
    def __init__(self, use_pretrained, feature_extracting, num_classes, LM=10, LM1=2,LM2=2, LM3=2, LM4=1):
        super(SiameseNetwork, self).__init__()
        
        self.LM=LM
        self.LM1=LM1
        self.LM2=LM2
        self.LM3=LM3
        self.LM4=LM4
        self.model_ft = Net()
        self.fc_mse = nn.Sequential(
            nn.Dropout(),
            nn.Linear(LM * LM1, LM2, bias = True),
            nn.ReLU(inplace = True),
            nn.Linear(LM3, LM4, bias = True)
        )


    def forward_once(self, x,zi=0,ouv=-1):
        self.zi=zi
        self.ouv=ouv
        
        output = self.model_ft(x)
        return output.view(output.size(zi), ouv)

    def forward(self, input1, input2,at=1):
        
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        self.at=at
        
        output = torch.cat((output1, output2), at)
        output = self.fc_mse(output)
        return output

In [ ]:




train_dataloader = DataLoader(AvesData,
                              shuffle = True,
                              num_workers = 2 ,
                              batch_size = 4)



In [ ]:


net = SiameseNetwork(True, True, 512).cuda()
criterion = nn.MSELoss().cuda()
optimizer = optim.Adam(net.parameters(), lr = 1e-4)
net.train()



In [ ]:
counter = []
loss_history_1 = [] 
loss_history_0 = [] 
iteration_number = 0

In [ ]:
#training
epocs

for epoch in range(0, epocs):
    
    for i, data in enumerate(train_dataloader, 0):
        print(data)
        
        imgAncla=data["spectograms"]["Ancla"][0]
        img1=data["spectograms"]["positive"][0]
        img0=data["spectograms"]["negative"][0]
        
        dif_0=data["labels"]["negative"]
        dif_1=data["labels"]["positive"]
        
        imgAncla,img0, img1, dif_1,dif_0 = imgAncla.cuda(), img0.cuda(), img1.cuda(), dif_1.cuda(), dif_0.cuda()
        optimizer.zero_grad()

        output_0 = net(imgAncla, img0)
        #dif? en dos diferentes optim()
        loss_0 = criterion(output_0, dif_0.float())
       
        loss_0.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        output_1 = net(imgAncla, img1)
    
        loss_1 = criterion(output_1, dif_1.float()) #dif? en dos diferentes optim()
    
        loss_1.backward()
       
        optimizer.step()
    print("Epoch number {}\n Current loss_1 {}\n".format(epoch, loss_1.item()))
    print("Epoch number {}\n Current loss_0 {}\n".format(epoch, loss_0.item()))
    iteration_number += 1
    counter.append(iteration_number)
    loss_history_1.append(loss_1.item())
    loss_history_0.append(loss_0.item())

In [ ]:
fig = plt.figure(figsize = (20, 10))
plt.plot(counter, loss_history_1, label='Positivos')
plt.plot(counter, loss_history_0, label='Negativos')
plt.title('MSE Positivos y Negativos')
plt.xlabel('Epochs')
plt.ylabel('Loss_1')
plt.rc('font', size = 15)
plt.legend()
plt.grid()
plt.show()
